# Pose visualizations VTTI airport
- colmap results for VTTI airport
- pose estimations, sparse map results

In [1]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go 
from scipy.spatial.transform import Rotation as R
from colmapParsingUtils import *
import copy

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt

import cv2

%load_ext autoreload
%autoreload 2
%autosave 180

/home/daniel-choate/miniconda3/envs/py39/lib/python3.9/site-packages/dash/dash.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, parse_version


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# Import colmap files

images_colmap = read_images_text('VTTI_data/images.txt')
cameras = read_cameras_text('VTTI_data/cameras.txt')
pts3d = read_points3D_text('VTTI_data/points3D.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'VTTI_data/images.txt'

In [ ]:
def get_pose_id_mod(id):
    """
    Get the pose transformation for a specific image id
    Input: Image ID
    Output: transform from camera to world coordinates
    """
    # Get quaternion and translation vector
    qvec = images_colmap[id].qvec
    tvec = images_colmap[id].tvec[:,None]
    # print(tvec)
    t = tvec.reshape([3,1])
    # Create rotation matrix
    Rotmat = qvec2rotmat(qvec) # Positive or negative does not matter
    # print("\n Rotation matrix \n", Rotmat)
    # Create 4x4 transformation matrix with rotation and translation
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    w2c = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(w2c)
    # self.im_pts_2d[imnum]['w2c'] = w2c
	# self.im_pts_2d[imnum]['c2w'] = c2w
    return w2c, c2w

In [ ]:
# Grab all poses 
# print(gnav.images_c)
poses = np.zeros([len(images_colmap),4,4])

for n in range(len(images_colmap)):
    # print(n)
    __, c2w = get_pose_id_mod(n+1)
    # print(c2w)
    poses[n] = c2w

print(poses)

In [ ]:
# Grab raw point cloud and rgb data from scene 

raw_pts = [pts3d[key].xyz for key in pts3d.keys()]
raw_rgb = [pts3d[key].rgb for key in pts3d.keys()]
# print(raw_pts)
scene_pts = np.vstack(raw_pts)
scene_rgb = np.vstack(raw_rgb)
# Normalize
scene_rgb = scene_rgb / 255
print(scene_pts)
print(scene_rgb)

In [ ]:
print(len(poses))

In [ ]:
# PLOT INITIAL POINT CLOUD AND POSES 
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud')#, width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5)

# loop through poses 
# for pose in poses:
for i in range(len(poses)):
    if (poses[i] == 0).all():
        print("Empty pose", i)
        continue
# for i in range(1):
    # print(i)
    homog_t = poses[i]
    # homog_t = pose
    # print(homog_t)
    s = i*.01
    # print(s)
    # if i in np.arange(106,135):
    if i in np.array([35, 40, 45, 50, 55, 60, 65, 70]):
        axes1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.25).transform(homog_t)
        vis.add_geometry(axes1)
    axes1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.1).transform(homog_t)
    # axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(scene_rgb)

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 1.5


# Run the visualizer
vis.run()
vis.destroy_window()


### FRAMES IN USE: 35, 40, 45, 50, 55, 60, 65, 70

- frame_30850.jpg
- frame_30900.jpg
- frame_30950.jpg
- frame_31000.jpg
- frame_31050.jpg
- frame_31100.jpg
- frame_31150.jpg
- frame_31200.jpg

see <vtti_gnav_demo.ipynb>

In [ ]:
print(images_colmap[135].name)

In [3]:
for elt in np.array([35, 40, 45, 50, 55, 60, 65, 70]):
    print(images_colmap[elt+1].name)

NameError: name 'images_colmap' is not defined

In [4]:
# Determining ground plane points 

gnd_pts_IDX = np.array([525, 532, 533, 534, 538, 539, 6178, 6180, 6188, 6190, 6193, 6192, 5385, 5386, 5388, 5389, 5390, 5391, 5392, 5393])

#5000 to 6000
arr2 = np.arange(5385, 5400)
arr1 = np.array([1])
arr3 = np.array([5391, 5392, 5393])
gnd_pts_TEST = np.concatenate((arr1, arr2))

In [ ]:
print(len(scene_pts))
print(gnd_pts_TEST)
print(scene_pts[gnd_pts_TEST])

In [ ]:
# PLOT INITIAL POINT CLOUD AND POSES 
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud')#, width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5)

scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(scene_rgb)

gnd_pts_t = o3d.geometry.PointCloud()
gnd_pts_t.points = o3d.utility.Vector3dVector(scene_pts[gnd_pts_TEST])
gnd_pts_t.paint_uniform_color([1,0,0])
# gnd_pts_t.colors = o3d.utility.Vector3dVector(scene_rgb[gnd_pts_TEST])

gnd_pts = o3d.geometry.PointCloud()
gnd_pts.points = o3d.utility.Vector3dVector(scene_pts[gnd_pts_IDX])
gnd_pts.paint_uniform_color([0,1,0])


vis.add_geometry(axes)
# vis.add_geometry(scene_cloud)
# vis.add_geometry(gnd_pts_t)
vis.add_geometry(gnd_pts)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2


# Run the visualizer
vis.run()
vis.destroy_window()

In [ ]:
# # GIF HELP

# # Capture frames for GIF
# frames = []
# num_frames = 30  # Adjust the number of frames
# angle_step = 180/num_frames


# for i in range(num_frames):
# 	# Rotate the view
#     view_control.rotate(angle_step, 0)  # (horizontal, vertical)

#     # vis.update_geometry(axis_orig) # Only if I move it myself?
#     vis.poll_events()
#     vis.update_renderer()

#     # Capture frame directly into memory
#     image = vis.capture_screen_float_buffer(False)
#     image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
#     frames.append(image_8bit)


# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, angle_step/5)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
#     view_control.rotate(-angle_step, -angle_step/5)  # (horizontal, vertical)

#     # vis.update_geometry(axis_orig) # Only if I move it myself?
#     vis.poll_events()
#     vis.update_renderer()

#     # Capture frame directly into memory
#     image = vis.capture_screen_float_buffer(False)
#     image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
#     frames.append(image_8bit)


# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# # Create GIF
# # Ensure frames are in the correct format
# frames = [frame.astype("uint8") for frame in frames]

# # Use imageio to save as GIF
# imageio.mimsave("test.gif", frames, fps=30, loop=0)  # Adjust fps if necessary